In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import math
import time, sys
import pickle
import timeit
#import wandb
import shutil
import pandas as pd
import keras.backend as K
from tensorflow.keras import layers, constraints

#os.environ["WANDB_API_KEY"] = ""
plt.ioff()
from keras.constraints import Constraint
import keras


In [2]:
# update_progress() : Displays or updates a console progress bar
## Accepts a float between 0 and 1. Any int will be converted to a float.
## A value under 0 represents a 'halt'.
## A value at 1 or bigger represents 100%
def update_progress(progress):
    barLength = 10 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), progress*100, status)
    sys.stdout.write(text)
    sys.stdout.flush()
#############################################
class D_ReLU(keras.layers.Layer):
    """ReLU"""
    def __init__(self):
        super(D_ReLU, self).__init__()

    def call(self, mu_in):
        mu_out = tf.nn.relu(mu_in)
        return mu_out
#############################################
class D_GeLU(keras.layers.Layer):
    def __init__(self):
        super(D_GeLU, self).__init__()
    def call(self, mu_in):  # mu_in = [50,17,64], Sigma_in= [50,17,64]
        mu_out = tf.nn.gelu(mu_in)  # [50,17,64]
        return mu_out  # [50,2,17,64], [50,2,17,64,64]
#############################################
class D_Softmax(keras.layers.Layer):
    def __init__(self):
        super(D_Softmax, self).__init__()
    def call(self, mu_in):
        mu_dim = tf.cast(tf.shape(mu_in)[-1], tf.float32)
        mu_out = tf.nn.softmax(mu_in)
        return mu_out

#############################################
class D_Dropout(keras.layers.Layer):
    def __init__(self, drop_prop):
        super(D_Dropout, self).__init__()
        self.drop_prop = drop_prop

    def call(self, mu_in, Training=None):
        # shape=[batch_size, seq length, embedding_dim]
        if Training:
            mu_out = tf.nn.dropout(mu_in, rate=self.drop_prop)  # [50,17,64] or [50, 10]
        else:
            mu_out = mu_in
        return mu_out  # [50,17,64], [50,17,64]
#############################################
class D_Dense(keras.layers.Layer):
    def __init__(self, units=32):
        '''
        Initialize the instance attributes
        '''
        super(D_Dense, self).__init__()
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight(name='kernel',
                                 shape=(input_shape[-1], self.units),
                                 initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05, seed=None),
                                 trainable=True)
    def call(self, inputs):
        return tf.matmul(inputs, self.w) #+ self.b

#############################################
class D_MLP(tf.keras.layers.Layer):
    def __init__(self, hidden_features, out_features, dropout_rate=0.1):
        super(D_MLP, self).__init__()
        self.dense1 = D_Dense(hidden_features)
        self.dense2 = D_Dense(out_features)
        self.dropout1 = D_Dropout(dropout_rate)
        self.gelu_1 = D_GeLU()

    def call(self, mu_in,training):
        mu_out = self.dense1(mu_in)
        mu_out = self.gelu_1(mu_out)
        mu_out = self.dropout1(mu_out,Training=training)
        mu_out = self.dense2(mu_out)
        mu_out = self.dropout1(mu_out, Training= training)
        return mu_out

#############################################
class D_LayerNorm(tf.keras.layers.Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(D_LayerNorm, self).__init__(**kwargs)

    def build(self, input_shape):
        self.gamma = self.add_weight(shape=input_shape[-1:],
                                     initializer=tf.keras.initializers.Ones(), name='gamma',trainable=True)
        self.beta = self.add_weight(shape=input_shape[-1:],
                                    initializer=tf.keras.initializers.Zeros(), name='beta', trainable=True)
        super(D_LayerNorm, self).build(input_shape)
    def call(self, x):
        mean = tf.math.reduce_mean(x, axis=-1, keepdims=True)
        std = tf.math.reduce_std(x, axis=-1, keepdims=True)
        # print( "mean of LN",mean.shape)
        # print("std of LN",std.shape)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape

class D_MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8, dropout = 0.1):
        super(D_MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads  # dimensionality of each attention head
        # create four dense layers (i.e., fully connected layers) that will be used in the attention mechanism.
        self.query_dense = D_Dense(embed_dim)
        self.key_dense = D_Dense(embed_dim)
        self.value_dense = D_Dense(embed_dim)
        self.combine_heads = D_Dense(embed_dim)
        self.dropout = D_Dropout(dropout)

    def attention(self, query, key, value, training):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        weights = self.dropout(weights, Training = training) #According to original VIT PAPER
        output = tf.matmul(weights, value)
        return output, weights
        '''
        This is a helper function that separates the num_heads attention heads
        from the input tensor x. It first reshapes x to split the last dimension
        into two dimensions of size num_heads and projection_dim, and
        then transposes the resulting tensor to move the num_heads dimension to the second position.
        '''
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])  # [50,2,17,32]
    def call(self, inputs,training):
        batch_size = tf.shape(inputs)[0]
        '''
        These lines apply dense layers to the input tensor inputs to get the key
        and value tensors, respectively. The key_dense and value_dense layers are
        initialized in the constructor of the class and are fully connected layers
        with an output size of embed_dim. The key and value tensors have the same
        shape as the inputs tensor except for last dimension, which is embed_dim.
           '''
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        '''
        These lines reshape the query, key, and value tensors into a tensor with
        shape (batch_size, num_heads, seq_len, projection_dim) using the
        separate_heads method. This method reshapes the last dimension of the input
         tensor into projection_dim and then reshapes the tensor into num_heads
         separate tensors along the second dimension.
         '''
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        '''
        This line applies the attention method to the query, key, and value tensors
        to compute the attention weights and the weighted sum of the value tensors.
        The attention method takes the query, key, and value tensors as inputs and
        returns the output tensor and the weights tensor.
           '''
        attention, weights = self.attention(query, key, value, training=training)  # [50,2,17,32]
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # [50,17,2,32]
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output
#############################################
class D_TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super(D_TransformerBlock, self).__init__()
        self.att = D_MultiHeadSelfAttention(embed_dim, num_heads, dropout)
        self.mlp = D_MLP(embed_dim * 2, embed_dim, dropout)
        # self.elu_1 = DGeLU()
        self.layernorm = D_LayerNorm(eps=1e-6)
        #self.dropout = D_Dropout(dropout)

    def call(self, inputs, training):
        inputs_norm = self.layernorm(inputs)
        attn_output = self.att(inputs_norm, training=training)
        #attn_output = self.dropout(attn_output, Training=training)
        out1 = attn_output + inputs
        out1_norm = self.layernorm(out1)
        mlp_output = self.mlp(out1_norm, training=training)
        #mlp_output = self.dropout(mlp_output, Training=training)
        return mlp_output + out1

In [3]:
class D_ViT(tf.keras.Model):
    def __init__(
            self,
            image_size=28,
            patch_size=4,
            num_layers=7,
            num_classes=10,
            embed_dim=64,
            num_heads=4,
            mlp_dim=64,
            channels=1,
            dropout=0.1,
            name=None,
            **kwargs
    ):
        super(D_ViT, self).__init__(**kwargs)
        num_patches = (image_size // patch_size) ** 2
        self.patch_dim = channels * (patch_size ** 2)

        self.image_size = image_size
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.num_heads = num_heads
        self.channels = channels
        self.dropout_rate = dropout
        self.mlp_dim = mlp_dim
        self.pos_emb = self.add_weight(shape=(1, num_patches + 1, embed_dim),name = "pos_emb")
        self.class_emb = self.add_weight(shape=(1, 1, embed_dim),name = "class_emb")
        self.patch_proj = D_Dense(embed_dim)

        self.enc_layers = [
            D_TransformerBlock(embed_dim, num_heads, mlp_dim, dropout)
            for _ in range(num_layers)
        ]

        self.mlp_head = D_MLP(mlp_dim, num_classes)
        self.softmax_out = D_Softmax()
        self.dropout = D_Dropout(dropout)

    def build(self, input_shape):
        # This ensures the model is built before loading weights
        super().build(input_shape)

    def initialize_weights(self):
        """Initialize the model by running a forward pass with dummy data."""
        dummy_input = tf.random.normal((1, self.image_size, self.image_size, self.channels))
        _ = self(dummy_input, training=False)

    def load_weights_safely(self, weights_path):
        """Safely load weights after ensuring model is initialized."""
        # Initialize the model if not already done
        if not self.built:
            self.initialize_weights()
        # Load the weights
        self.load_weights(weights_path)

    def extract_patches(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID")
        patches = tf.reshape(patches, [batch_size, -1, self.patch_dim])
        return patches

    def call(self, x, training=None):
        batch_size = tf.shape(x)[0]
        patches = self.extract_patches(x)
        x = self.patch_proj(patches)
        class_emb = tf.broadcast_to(self.class_emb, [batch_size, 1, self.embed_dim])
        x = tf.concat([class_emb, x], axis=1)
        x = x + self.pos_emb
        x = self.dropout(x, Training=training)  #According to original VIT Paper
        for layer in self.enc_layers:
            x = layer(x, training = training)
        x = self.mlp_head(x[:, 0],training=training)
        x = self.softmax_out(x)
        return x

In [12]:
def main_function(image_size=28, patch_size=4, num_layers=5, num_classes=10, embed_dim=32, num_heads=4, mlp_dim=32,
                  channels=1, drop_prob=0.01, batch_size=50, epochs=5, lr=0.001, lr_end=0.00001,
                  Targeted=False, Random_noise=False, gaussain_noise_std=0.3, Training=True, Testing=False,
                  ):

    PATH = './trans_epoch_{}/'.format(epochs)
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = tf.expand_dims(x_train, -1)
    x_test = tf.expand_dims(x_test, -1)
    one_hot_y_train = tf.one_hot(y_train.astype(np.float32), depth=num_classes)
    one_hot_y_test = tf.one_hot(y_test.astype(np.float32), depth=num_classes)
    tr_dataset = tf.data.Dataset.from_tensor_slices((x_train, one_hot_y_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, one_hot_y_test)).batch(batch_size)

    trans_model = D_ViT(image_size=image_size, patch_size=patch_size, num_layers=num_layers, num_classes=num_classes,
                          embed_dim=embed_dim, num_heads=num_heads, mlp_dim=mlp_dim,
                          channels=channels, dropout=drop_prob, name='det_trans')

    num_train_steps = epochs * int(x_train.shape[0] / batch_size)
    #    step = min(step, decay_steps)
    #    ((initial_learning_rate - end_learning_rate) * (1 - step / decay_steps) ^ (power) ) + end_learning_rate
    learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=lr,
                                                                     decay_steps=num_train_steps,
                                                                     end_learning_rate=lr_end, power=1.)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn , clipvalue=1.0)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

    @tf.function  # Make it fast.
    def train_on_batch(x, y):
        with tf.GradientTape() as tape:
            out = trans_model(x, training=True)
            trans_model.trainable = True
            loss = loss_fn(y, out)

        gradients = tape.gradient(loss, trans_model.trainable_weights)
        gradients = [(tf.where(tf.math.is_nan(grad), tf.constant(1.0e-5, shape=grad.shape), grad)) for grad in gradients]
        gradients = [(tf.where(tf.math.is_inf(grad), tf.constant(1.0e-5, shape=grad.shape), grad)) for grad in gradients]
        optimizer.apply_gradients(zip(gradients, trans_model.trainable_weights))
        return loss, out, gradients
    @tf.function
    def validation_on_batch(x, y):
        trans_model.trainable = False
        out = trans_model(x, training=False)
        vloss = loss_fn(y, out)

        return vloss, out

    @tf.function
    def test_on_batch(x, y):
        trans_model.trainable = False
        out = trans_model(x, training=False)
        return out

    if Training:
        trans_model.initialize_weights()
        if os.path.exists(PATH):
            shutil.rmtree(PATH)
        os.makedirs(PATH)
        # wandb.init(entity="transformer",
        #            project="Trans_mnist_epochs_{}_layer_{}_lr_{}_batch_size_{}_dimension_{}_patch_size_{}_head_{}_drop_{}".format(
        #                epochs, num_layers, lr, batch_size, embed_dim, patch_size, num_heads,drop_prob))

        train_acc = np.zeros(epochs)
        valid_acc = np.zeros(epochs)
        train_err = np.zeros(epochs)
        valid_error = np.zeros(epochs)
        start = timeit.default_timer()
        for epoch in range(epochs):
            print('Epoch: ', epoch+1, '/' , epochs)
            acc1 = 0
            acc_valid1 = 0
            err1 = 0
            err_valid1 = 0
            tr_no_steps = 0
            va_no_steps = 0
            #-------------Training--------------------
            acc_training = np.zeros(int(x_train.shape[0] / (batch_size)))
            err_training = np.zeros(int(x_train.shape[0] / (batch_size)))
            for step, (x, y) in enumerate(tr_dataset):
                update_progress(step/int(x_train.shape[0]/(batch_size)) )
                loss, mu_out, gradients = train_on_batch(x, y)
                err1+= loss.numpy()
                corr = tf.equal(tf.math.argmax(mu_out, axis=1),tf.math.argmax(y,axis=1))
                accuracy = tf.reduce_mean(tf.cast(corr,tf.float32))
                acc1+=accuracy.numpy()
                if step % 100 == 0:
                    print('\n gradient', np.mean(gradients[0].numpy()))
                    print("\n Step:", step, "Loss:" , float(err1/(tr_no_steps + 1.)))
                    print("Total Training accuracy so far: %.3f" % float(acc1/(tr_no_steps + 1.)))
                tr_no_steps+=1
                # wandb.log({"Total Training Loss": loss.numpy() ,
                #             "Training Accuracy per minibatch": accuracy.numpy() ,
                #             "gradient per minibatch": np.mean(gradients[0]),
                #             'epoch': epoch
                #    })
            train_acc[epoch] = acc1/tr_no_steps
            train_err[epoch] = err1/tr_no_steps
            print('Training Acc  ', train_acc[epoch])
            print('Training error  ', train_err[epoch])
            #---------------Validation----------------------
            for step, (x, y) in enumerate(val_dataset):
                update_progress(step / int(x_test.shape[0] / (batch_size)) )
                total_vloss, mu_out = validation_on_batch(x, y)
                err_valid1+= total_vloss.numpy()
                corr = tf.equal(tf.math.argmax(mu_out, axis=-1),tf.math.argmax(y,axis=-1))
                va_accuracy = tf.reduce_mean(tf.cast(corr,tf.float32))
                acc_valid1+=va_accuracy.numpy()

                if step % 100 == 0:
                    print("Step:", step, "Loss:", float(total_vloss))
                    print("Total validation accuracy so far: %.3f" % va_accuracy)
                va_no_steps+=1
                # wandb.log({"Total Validation Loss": total_vloss.numpy() ,
                #               "Validation Acuracy per minibatch": va_accuracy.numpy()
                #                })
            valid_acc[epoch] = acc_valid1/va_no_steps
            valid_error[epoch] = err_valid1/va_no_steps
            stop = timeit.default_timer()
            if np.max(valid_acc) == valid_acc[epoch]:
                trans_model.save_weights(PATH+'Trans_model_best.weights.h5')
                #trans_model.save(PATH + 'Deterministic_cnn_model_best.keras')

            #trans_model.save_weights(PATH + 'vdp_trans_model')
            # wandb.log({"Average Training Loss":  train_err[epoch],
            #            "Average Training Accuracy": train_acc[epoch],
            #            "Average Validation Loss": valid_error[epoch],
            #            "Average Validation Accuracy": valid_acc[epoch],
            #            'epoch': epoch
            #           })
            print('Total Training Time: ', stop - start)
            print('Training Acc  ', train_acc[epoch])
            print('Validation Acc  ', valid_acc[epoch])
            print('------------------------------------')
            print('Training error  ', train_err[epoch])
            print('Validation error  ', valid_error[epoch])
        #-----------------End Training--------------------------
        trans_model.save_weights(PATH+'Trans_model_last.weights.h5')
        if (epochs > 1):
            fig = plt.figure(figsize=(15,7))
            plt.plot(train_acc, 'b', label='Training acc')
            plt.plot(valid_acc,'r' , label='Validation acc')
            plt.ylim(0, 1.1)
            plt.title("Transformer on MNIST Data")
            plt.xlabel("Epochs")
            plt.ylabel("Accuracy")
            plt.legend(loc='lower right')
            plt.savefig(PATH + 'Trans_on_MNIST_Data_acc.png')
            plt.close(fig)

            fig = plt.figure(figsize=(15,7))
            plt.plot(train_err, 'b', label='Training error')
            plt.plot(valid_error,'r' , label='Validation error')
            plt.title("Transformer on MNIST Data")
            plt.xlabel("Epochs")
            plt.ylabel("Error")
            plt.legend(loc='upper right')
            plt.savefig(PATH + 'Trans_on_MNIST_Data_error.png')
            plt.close(fig)

        f = open(PATH + 'training_validation_acc_error.pkl', 'wb')
        pickle.dump([train_acc, valid_acc, train_err, valid_error], f)
        f.close()

        textfile = open(PATH + 'Related_hyperparameters.txt','w')
        textfile.write(' Input Dimension : ' + str(image_size))
        textfile.write('\n Hidden units : ' + str(mlp_dim))
        textfile.write('\n Number of Classes : ' + str(num_classes))
        textfile.write('\n No of epochs : ' + str(epochs))
        textfile.write('\n Initial Learning rate : ' + str(lr))
        textfile.write('\n Ending Learning rate : ' + str(lr_end))
        textfile.write('\n batch size : ' + str(batch_size))
        textfile.write("\n---------------------------------")
        if Training:
            textfile.write('\n Total run time in sec : ' + str(stop - start))
            if (epochs == 1):
                textfile.write("\n Averaged Training  Accuracy : " + str(train_acc))
                textfile.write("\n Averaged Validation Accuracy : " + str(valid_acc))

                textfile.write("\n Averaged Training  error : " + str(train_err))
                textfile.write("\n Averaged Validation error : " + str(valid_error))
            else:
                textfile.write("\n Averaged Training  Accuracy : " + str(np.mean(train_acc[epoch])))
                textfile.write("\n Averaged Validation Accuracy : " + str(np.mean(valid_acc[epoch])))

                textfile.write("\n Averaged Training  error : " + str(np.mean(train_err[epoch])))
                textfile.write("\n Averaged Validation error : " + str(np.mean(valid_error[epoch])))
        textfile.write("\n---------------------------------")
        textfile.write("\n--------------------------------")
        textfile.close()

    #-------------------------Testing-----------------------------
    if Testing:
        test_path = 'test_results/'
        if Random_noise:
            print(f'Random Noise: {gaussain_noise_std}')
            test_path = 'test_results_random_noise_{}/'.format(gaussain_noise_std)
        full_test_path = PATH + test_path
        if os.path.exists(full_test_path):
            # Remove the existing test path and its contents
            shutil.rmtree(full_test_path)
        os.makedirs(PATH + test_path)
        trans_model.load_weights_safely(PATH + 'Trans_model_best.weights.h5')

        test_no_steps = 0
        true_x = np.zeros([int(x_test.shape[0] / (batch_size)), batch_size, image_size, image_size, 1])
        true_y = np.zeros([int(x_test.shape[0] / (batch_size)), batch_size, num_classes])
        mu_out_ = np.zeros([int(x_test.shape[0] / (batch_size)), batch_size, num_classes])
        acc_test = np.zeros(int(x_test.shape[0] / (batch_size)))
        for step, (x, y) in enumerate(val_dataset):
            update_progress(step / int(x_test.shape[0] / (batch_size)))
            true_x[test_no_steps, :, :, :, :] = x
            true_y[test_no_steps, :, :] = y
            if Random_noise:
                noise = tf.random.normal(shape=[batch_size, image_size, image_size, 1], mean=0.0,
                                         stddev=gaussain_noise_std, dtype=x.dtype)
                x = x + noise
            mu_out = test_on_batch(x, y)
            mu_out_[test_no_steps, :, :] = mu_out
            corr = tf.equal(tf.math.argmax(mu_out, axis=1), tf.math.argmax(y, axis=1))
            accuracy = tf.reduce_mean(tf.cast(corr, tf.float32))
            acc_test[test_no_steps] = accuracy.numpy()
            if step % 100 == 0:
                print("Total running accuracy so far: %.3f" % acc_test[test_no_steps])
            test_no_steps += 1

        test_acc = np.mean(acc_test)
        print('\nTest accuracy : ', test_acc)


        pf = open(PATH + test_path + 'info.pkl', 'wb')
        pickle.dump([mu_out_, true_x, true_y, test_acc], pf)
        pf.close()


        var = np.zeros([int(x_test.shape[0] / (batch_size)), batch_size])
        if Random_noise:
            snr_signal = np.zeros([int(x_test.shape[0] / (batch_size)), batch_size])
            for i in range(int(x_test.shape[0] / (batch_size))):
                for j in range(batch_size):
                    noise = tf.random.normal(shape=[image_size, image_size, 1], mean=0.0, stddev=gaussain_noise_std,
                                             dtype=x.dtype)
                    snr_signal[i, j] = 10 * np.log10(np.sum(np.square(true_x[i, j, :, :, :])) / np.sum(np.square(noise)))

            print('SNR', np.mean(snr_signal))

        textfile = open(PATH + test_path + 'Related_hyperparameters.txt', 'w')
        textfile.write(' Input Dimension : ' + str(image_size))
        textfile.write('\n Number of Classes : ' + str(num_classes))
        textfile.write('\n No of epochs : ' + str(epochs))
        textfile.write('\n Initial Learning rate : ' + str(lr))
        textfile.write('\n Ending Learning rate : ' + str(lr_end))
        textfile.write('\n batch size : ' + str(batch_size))
        textfile.write("\n---------------------------------")
        textfile.write("\n Test Accuracy : " + str(test_acc))
        textfile.write("\n---------------------------------")
        if Random_noise:
            textfile.write('\n Random Noise std: ' + str(gaussain_noise_std))
            textfile.write("\n SNR: " + str(np.mean(snr_signal)))
        textfile.write("\n---------------------------------")
        textfile.close()


In [10]:
if __name__ == '__main__':
     main_function()

Epoch:  1 / 5
Percent: [----------] 0.0% 
 gradient 3.6862536e-06

 Step: 0 Loss: 2.3023521900177
Total Training accuracy so far: 0.160
Percent: [----------] 4.166666666666666% 
 gradient -2.3734226e-05

 Step: 50 Loss: 2.211318254470825
Total Training accuracy so far: 0.170
Percent: [#---------] 8.333333333333332% 
 gradient -3.3725955e-05

 Step: 100 Loss: 2.0832881927490234
Total Training accuracy so far: 0.214
Percent: [#---------] 12.5% 
 gradient 6.6263788e-06

 Step: 150 Loss: 2.0214943885803223
Total Training accuracy so far: 0.234
Percent: [##--------] 16.666666666666664% 
 gradient 2.3134662e-05

 Step: 200 Loss: 1.9800227880477905
Total Training accuracy so far: 0.250
Percent: [##--------] 20.833333333333336% 
 gradient 5.496631e-05

 Step: 250 Loss: 1.9393558502197266
Total Training accuracy so far: 0.263
Percent: [##--------] 25.0% 
 gradient 1.3048919e-05

 Step: 300 Loss: 1.8924305438995361
Total Training accuracy so far: 0.280
Percent: [###-------] 29.166666666666668% 


In [13]:
if __name__ == '__main__':
     main_function(Training=False , Testing=True)

Percent: [----------] 0.0% Total running accuracy so far: 0.940
Percent: [#####-----] 50.0% Total running accuracy so far: 1.000
Percent: [##########] 99.5% 
Test accuracy :  0.9460999989509582
